# Rcupcake: Case study on Simon Simplex Collection database
We will use this example to show the functionality of the Rcupcake package when both, genotype and phenotype variables are present in the dataset.

## Introduction
Nowadays, autism spectrum disorder (ASD) affects 1 in 68 children, being one of the fastest-growing developmental disabilities. Autism has been associated with a wide range of disorders, and many genes are related to it. However, the nature and extent of the comorbidities of autism at the population level need to be determined.

Simon Simplex Collection (SSC) is an autism dataset that contains 2600 simplex families with one proband who has been diagnosed with autism, while the father, the mother and a sibling has not. All information collected during two years have been integrated in one giant system gathering genetic and clinical data.

The Rcupcake package allows retrieving the SSC data for a set of phenotypes regarding the presence or not of at least one de novo mutation in a specific gene. In this case study, we will describe how to analyze a set of phenotypes regarding one specific gene; the CHD8 gene (Binary variable: at least one de novo mutation in the gene CHD8). The phenotypes of interest for this case study will be, the head circumference, known to be associated with de novo mutations in CHD8 gene, and loss of language, facial expression and compulsions rituals (known to be associated with ASD). 


## Main Goal
The goal of the study is to investigate whether different phenotypes variables and phenotype co-occurrences are associated with de novo mutation in CHD8 gene. More specifically, we will answer the following questions:
1. How can we query SSC data through the BD2K PIC-SURE API RESTful API using R?
2. What are the main demographic characteristics of our study population?
3. What are the main phenotypic characteristics of our study population?
4. Are the phenotype variables statistically associated with CHD8 mutation
5. What are the phenotype co-occurrences of patients with de novo CHD8 mutation?
    
## Implementation

### Rcupcake package
The Rcupcake package enables the user to:
1. Query through BD2K PIC-SURE RESTful API
2. Explore the data from a demographic point of view
3. Analyze the phenotype variables according to their different values, in terms of prevalence and co-occurrence
4. Perform univariate analyses
5. Describe the phenotype co-occurrence according to the patient exposure, such as having or not one de novo mutation in a specific gene.
    
## Installation
The package `Rcupcake` is provided through GitHub. To install `Rcupcake` the user must type the following commands in an R session:

In [ ]:
library(devtools)
install_github("hms-dbmi/Rcupcake", force =TRUE)
library(Rcupcake)

## Questions that can be answered using `Rcupcake`


### Workflow
In order to answer the different questions, the steps will be:
* Data retrieval 
* Descriptive demographics analysis 
* Descriptive phenotypic analysis 
* Univariate analysis. 
* Phenotype co-occurrence: patients with at least one de novo CHD8 mutation.


  <section data-markdown>
                    <img src="./sscWorkflow.png" | width =300/>
                </section>

#  1. How can we query SSC data through the BD2K PIC-SURE RESTful API using `Rcupcake`?
We first select our study population: the subset of patients with available data regarding the variable CHD8 gene (Binary variable: at least one de novo mutation in the gene CHD8). Then, we select the variables that we want to retrieve for our analysis.
* CHD8 mutation 
* Age 
* Gender
* Head Circunference 
* Loss Language 
* Facial Expression 
* Compulsions current

## 1.1 Start session
The first step in order to retrieve the data of interest is to start the session. `start.session` function of the `Rcupcake` package establishes the connection to the database. As input we need to determine the URL of the database (https://ssc.hms.harvard.edu/) and the key to access the data. In order to access the SSC data, an authorization is required.


In [ ]:
sessionEx <- start.session( 
              url         = "https://ssc.hms.harvard.edu/", 
              apiKey      = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
              )
sessionEx

Once the connection has been established the output message will be: _'Start Session: success'_. Otherwise the message: _'Start Session: failed. Please revise your url and apiKey'_ will appear


## 1.2 Get children paths
The second step consists of retrieving the paths of the location of each variable of interest. This step will allow to create the JSON query to retrieve the data. For this case study, we know that age and gender variables are under the Demographics folder, the CHD8 gene information is under the "SSC_wigler_mutations" folder and the phenotypes of interest are under the "ssc_commonly_used" and "adi_r" path.
 
In order to retrieve the paths, we apply the `get.children` function, determining for each case, the URL of the database (https://ssc.hms.harvard.edu/) and the field name to access the data. At the end we will create one unique vector (sscVector) with the three previous ones.

In [ ]:
sscAdirPath <- get.children( 
   url         = "https://ssc.hms.harvard.edu/", 
   fieldname   = "/ssc/Demo/SFARI_Simplex_Collection_v15/SFARI_Simplex_Collection_v15/Clinical/adi_r/"
  )

sscDemographicPath <- get.children( 
   url         = "https://ssc.hms.harvard.edu/", 
   fieldname   = "/ssc/Demo/SFARI_Simplex_Collection_v15/SFARI_Simplex_Collection_v15/Clinical/Demographics/"
  )

sscCommonPath <- get.children( 
   url         = "https://ssc.hms.harvard.edu/", 
   fieldname   = "/ssc/Demo/SFARI_Simplex_Collection_v15/SFARI_Simplex_Collection_v15/Clinical/ssc_commonly_used/"
  )

sscMutationPath <- get.children( 
   url         = "https://ssc.hms.harvard.edu/", 
   fieldname   = "/ssc/Demo/SFARI_Simplex_Collection_v15/SFARI_Simplex_Collection_v15/SSC_wigler_mutations/"
  )

sscVector <- c( sscAdirPath, sscDemographicPath, sscCommonPath, sscMutationPath)
head(sscVector)


## 1.3 Build the JSON query

Once the paths have been gathered, we can create the JSON query to retrieve the variables of interest. We apply `my.query` function. The `my.query` function uses as input the URL of the database (https://ssc.hms.harvard.edu/), the vector with the `get.children` function output and a vector with the variables of interest.

__Note that the first variable of the `myfield` vector will be the one that will be used for where clause. In this example we want the subset of patients that have available data regarding the CHD8 de novo mutation, for this reason the CHD8 variable is the first item of the vector.__

In [ ]:
queryExample <- my.query( myfields  = "CHD8|SEX_CD|AGE_IN_YEARS_NUM|head_circumference|q70_compulsions_current|q57_facial_expressions_current|q11_loss_language", 
                          myvector  = sscVector, 
                          url       = "https://ssc.hms.harvard.edu/"
              )
queryExample

## 1.4 Run the query

The last step to retrieve the data consists of applying the `my.data` function to the JSON query built previously. Given the URL of the database (https://ssc.hms.harvard.edu/) and the JSON query, it generates a data.frame object with the query output. We have to select the output format and the path where the output will be located. Remember that by default it will be saved in the working directory.

In [ ]:
query <- my.data( 
    query          = queryExample,
    url            = "https://ssc.hms.harvard.edu/", 
    responseFormat = "CSV", 
    outputPath     = "./dataSSC.txt")

In [ ]:
head(query)

## 1.5 Transform the data.frame in `cupcakeData` object

Once the data is in a data.frame format, it has to be transformed into a `cupcakeData` class object in order to be analyzed with the `Rcupcake` package. Note that our example data contains:
* 3 demographic variables: patient_id, Gender and Age. 
* 4 phenotype variables: head circumference, current compulsion, facial expression and loss of language.
* 1 variation variable (here a mutation): CHD8

In order to transform the data.frame into a `cupcakeData` object, we apply the `dataframe2cupcake` function. We need to specify:
* `input`: that determines the file with the complete path where the required input file is located.
* `phenotypes`: vector that contains phenotype variables, separated by "|".
* `variants`: vector that contains the names of variations, separated by "|".
* `age`: vector that contains the age variable name.
* `gender`: vector that contains the gender variable name.

In [ ]:
data2b2 <- dataframe2cupcake( input = "./dataSSC.txt",
                              age        = "AGE",
                              gender     = "SEX_CD",
                              phenotypes = "head_circumference|q70_compulsions_current|q57_facial_expressions_current|q11_loss_language",
                              variants   = "CHD8",
                              verbose    = TRUE)
data2b2

The data2b2 object generated is a `cupcakeData` object that shows the number of patients (2,509 in this case), the number of phenotypes we are studying (4 in this case) and the number of variations (1 in this case). In order to see how the data looks like, we can apply the `extract` function as follows.

In [ ]:
head( extract( data2b2 ) )

# 2. Descriptive demographics analysis with __Rcupcake__


Now that we have extracted the data, we can start our analysis. We will first analyze the demographic variables. The function `demographic.summary` describes the demographic characteristics (sex, age) of our study population
Using as input:
* `input`: the data2b2 object that we have generated applying the `dataframe2cupcake` function.
* `maleCode`: the symbol which denotes males, in this case `MALE`
* `femaleCode`: the symbol which denotes females, in this case `FEMALE`

As output we retrieve:
* A barplot with the age distribution of the whole study population. We can see that our population ages range from 4 to 18 years old.
* A pie chart that represents the gender distribution. A boxplot that represents the age distribution by gender.

In [ ]:
demographic.summary (input       = data2b2, 
                    maleCode    = "MALE", 
                    femaleCode  ="FEMALE")
summary(as.numeric(data2b2@iresult$Age))

# 3. Descriptive phenotypic analysis with __Rcupcake__

The function `phenotype.summary` performs a descriptive analysis of the phenotypic variables for the whole study population in general, and according to the status of one selected variation.

The `phenotype.summary` function requires 2 arguments:
* `input`: the data2b2 object, that is a cupcakeData object (`dataframe2cupcake` function output – see above).
* `variation`: determines the variation of interest, in this case the mutation of interest ("CHD8").

Although the function can show the results as figures and tables, for this case study, we will only retrieve the table for this case study. We will set the showTable argument to `TRUE` and the showFigure one to `FALSE`.


In [ ]:
kable(phenotype.summary( input      = data2b2,
                         variation  = "CHD8", 
                         showTable  = TRUE, 
                         showFigure = FALSE), 
      format = "rst")

# 4. Univariate analysis with __Rcupcake__

The Rcupcake package also allows to perform a comparison two by two of a pair of variables in the `cupcakeData` object by applying the `comparison2b2` function. Applying this function we can easily see that:
* head circumference and age are significantly associated (p-value < 2.2e-16), as expected
* head circumference and de novo CHD8 mutation are significantly (p-value = 0.007204)


In [ ]:
comparison2b2( 
    input     = data2b2, 
    variable1 = "P.head_circumference",
    variable2 = "Age"
)

In [ ]:
comparison2b2( 
    input     = data2b2, 
    variable1 = "V.CHD8",
    variable2 = "P.head_circumference"
)

# 5. Phenotypes co-occurrence and prevalence using __Rcupcake__: patients with at least one de novo mutation in CHD8 gene

Finally, we will analyze the co-occurrence between the phenotypes according to CHD8 mutation status. We will apply the `co.occurrence` function that identifies all the possible phenotype co-occurrence and quantify them according to five different quantification measures (Fisher test, comorbidity score, relative risk, Pearson correlation and odds ratio).

__Note that according to the type of study, the user can select the co-occurrence measures needed as well as the cut-off value. As a result of the phenotype co-occurrence analysis, a table containing the different-estimated measurements is obtained.__

The `co.occurrence` function requires 6 arguments:
* `input`: the `cupcakeData` object, the data2b2 in this case.
* `pth`: the path where the file with the phenotype values generated previously is located.
* `aggregate`: if all the possible phenotype values want to be considered individually, aggregate must be `FALSE`. Otherwhise, the phenotype values should be manually completed by the user (column yes/no) and aggregate must be `TRUE`. In our case, we will set this argument as `FALSE`.
* `ageRange`: determines what is the age range of interest for performing the analysis. In our case, we will select all the patients, so we set from 4 to 18, 
* `gender`: determine what is the gender of interest for the co-occurrence analysis, in this case "ALL", because we do not want to distinguish between male and female.
* `variation`: determine the variation of interest and its value, in this case CHD8 and yes, because we are interested in those patients that have at least one de novo mutation in CHD8 gene.

The `co.occurrence` function output is a `cupcakeResults` object. This object shows us the conditions we have selected for the analysis (male and female from 4 to 18 years old and with at least one de novo CHD8 mutation). It also shows us a summary of the phenotype co-occurrence results, the prevalence of our disease of interest, the number of patients that have at least one de novo CHD8 mutation, the range of co-occurrence measures and the number of comorbidities (12 in our case).


In [ ]:
comorCHD8yes <- co.occurrence( 
              input         = data2b2,
              pth           = ".",
              aggregate    = FALSE, 
              ageRange      = c(4,18),
              gender        = "ALL", 
              variation      = c("CHD8", "yes")
              )
comorCHD8yes

If we want to retrieve the table with the specific results for each phenotype co-occurrence, we can apply the `extract` function to the `cupcakeResults` as showed here:

In [ ]:
head( extract( comorCHD8yes ) )

We can visualize the results with the `cooc.heatmap` and `cooc.network` functions. For this example, we will apply the `cooc.heatmap` function to visualize those phenotype co-occurrences that appear in at least 10% of the patients.

The `cooc.heatmap` function requires 3 arguments:
* `input`: the `comorCHD8yes` object
* `representedVariable`: the co-occurrence measurement
* `variableCutOff`: the numeric value of the cutOff

The `cooc.heatmap` function output is a heatmap. Blue color represents the lower values, yellow color represents the upper values. The representedVariable value is shown in each cell.

In [ ]:
heatmapCHD8yes <- cooc.heatmap( input               = comorCHD8yes,
                                representedVariable = "PercentagePhenoAB", 
                                variableCutOff      = 10, 
                                lowColor            = "blue", 
                                highColor           = "yellow")

heatmapCHD8yes

The same analysis can be done to the group of patients that do not have the novo mutation in the CHD8 gene.

In [ ]:
comorCHD8no <- co.occurrence( 
              input        = data2b2,
              pth          = ".",
              aggregate    = FALSE, 
              ageRange     = c(4,18),
              gender       = "ALL", 
              variation    = c("CHD8", "no")
              )
comorCHD8no

In [ ]:
heatmapCHD8no <- cooc.heatmap( input                = comorCHD8no,
                                representedVariable = "PercentagePhenoAB", 
                                variableCutOff      = 10, 
                                lowColor            = "blue", 
                                highColor           = "yellow")

heatmapCHD8no